In [1]:
import torch
from pcdet.ops.iou3d_nms import iou3d_nms_cuda
torch.ops.load_library(".\iou3d_nms_cuda.cp310-win_amd64.pyd")

In [2]:
def nms_gpu(boxes, scores, thresh, pre_maxsize=None):
    """
    :param boxes: (N, 7) [x, y, z, dx, dy, dz, heading]
    :param scores: (N)
    :param thresh:
    :return:
    """
    
    assert boxes.shape[1] == 7
    order = scores.sort(0, descending=True)[1]
    if pre_maxsize is not None:
        order = order[:pre_maxsize]

    boxes = boxes[order].contiguous()
    keep = torch.zeros(boxes.size(0)).int()

    num_out =iou3d_nms_cuda.nms_gpu(boxes, keep, thresh)

    return order[keep[:num_out].long().cuda()].contiguous()

def outer_nms_gpu(arg1, arg2):
    return nms_gpu(arg1, arg2, 0.01)

In [3]:
keep = torch.load("./keep.pt")
boxes = torch.load("./boxes.pt")
scores = torch.load("./scores.pt")
thresh = torch.load("./thresh.pt")
print(keep)
print(boxes)
print(thresh)
print(scores)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=torch.int32)
tensor([[  8.6952,  -1.8287,  -0.6740,  ...,   0.6650,   1.8718,   4.7095],
        [  1.3170,   4.3007,  -0.7707,  ...,   0.7244,   1.7644,   3.9951],
        [  1.3303,   4.2959,  -0.7686,  ...,   0.7037,   1.7561,   6.9227],
        ...,
        [ 21.8648, -20.8127,  -0.5143,  ...,   0.6068,   1.8300,   1.5008],
        [  2.1701,   3

In [4]:
traced_module = torch.jit.script(outer_nms_gpu, (boxes[0:100], scores[0:100]))
print(outer_nms_gpu(boxes, scores))
print(traced_module(boxes, scores))


# print(outer_nms_gpu((boxes, scores)))


d:\anaconda3\envs\env_openpcdet_test_cu118\lib\site-packages\torch\jit\_script.py:1241: UserWarning: `optimize` is deprecated and has no effect. Use `with torch.jit.optimized_execution() instead
  warnings.warn(


RuntimeError: 
Python builtin <built-in method nms_gpu of PyCapsule object at 0x000001B5937CAC40> is currently not supported in Torchscript:
  File "C:\Users\test111\AppData\Local\Temp\ipykernel_23320\580425827.py", line 17
    keep = torch.zeros(boxes.size(0)).int()

    num_out =iou3d_nms_cuda.nms_gpu(boxes, keep, thresh)
             ~~~~~~~~~~~~~~~~~~~~~~ <--- HERE

    return order[keep[:num_out].long().cuda()].contiguous()
'nms_gpu' is being compiled since it was called from 'outer_nms_gpu'
  File "C:\Users\test111\AppData\Local\Temp\ipykernel_23320\580425827.py", line 22
def outer_nms_gpu(arg1, arg2):
    return nms_gpu(arg1, arg2, 0.01)
           ~~~~~~~~~~~~~~~~~~~~~~~~ <--- HERE


In [ ]:
print(keep)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=torch.int32)
